# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [39]:
# import libraries
from sqlalchemy import create_engine

import re

import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier


import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score


import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/azatgabdolla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/azatgabdolla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/azatgabdolla/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import os

os.getcwd()

'/Users/azatgabdolla/Documents/edu/Udacity/de'

In [3]:
import os

if os.getcwd() == '/Users/azatgabdolla/Documents/edu/Udacity/de/notebook':
    pass
else:
    os.chdir(os.getcwd() +'/notebook')

In [4]:
os.getcwd()

'/Users/azatgabdolla/Documents/edu/Udacity/de/notebook'

In [9]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///project_database.db')
    df = pd.read_sql_table('df_1', con = engine) 
    X = df.message.values
    Y = df.iloc[:, range(4,df.shape[1])]
    Y = Y.loc[:, Y.nunique() > 1]
    return X,Y

In [10]:
X,Y = load_data()

In [43]:
engine = create_engine('sqlite:///project_database.db')
df = pd.read_sql_table('df_1', con = engine)

### 2. Write a tokenization function to process your text data

In [72]:
stopwords = stopwords.words('english')

# from collections import Counter
# stopwords_dict = Counter(stopwords_x)


In [11]:
def tokenize(text):
    
    '''
    - normalize case and remove punctuation
    - tokenize text
    - lemmatize andremove stop words within a loop
    
    '''
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        if tok not in stopwords:
            clean_tok = lemmatizer.lemmatize(tok).strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [75]:
pipeline = Pipeline(steps = [('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                        ('mclf', MultiOutputClassifier( LogisticRegression() 
                                                          )) ] )

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)


In [77]:
model = pipeline.fit(X_train, y_train)

In [78]:
Y['transport'].value_counts()

0    24827
1     1201
Name: transport, dtype: int64

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
# use this function instead

def evaluate_model(model, X_test, y_test, columns = Y.columns):
    
    '''
    - predict on the test data
    - classification report
    '''
    
    
    y_pred = model.predict(X_test)
    y_test = np.array(y_test)

    for i, col in enumerate(columns): 
            print('---------------------------------##-----------------')
            print(col)
            print(classification_report(y_test[:,i], y_pred[:,i]))

In [99]:
evaluate_model(model, X_test, y_test, columns = Y.columns)

---------------------------------
related
              precision    recall  f1-score   support

           0       0.77      0.41      0.53      1569
           1       0.84      0.96      0.89      4938

    accuracy                           0.83      6507
   macro avg       0.80      0.68      0.71      6507
weighted avg       0.82      0.83      0.81      6507

---------------------------------
request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5403
           1       0.82      0.55      0.66      1104

    accuracy                           0.90      6507
   macro avg       0.87      0.76      0.80      6507
weighted avg       0.90      0.90      0.89      6507

---------------------------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   m

/Users/azatgabdolla/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/azatgabdolla/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/azatgabdolla/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      4693
           1       0.85      0.67      0.75      1814

    accuracy                           0.88      6507
   macro avg       0.87      0.81      0.83      6507
weighted avg       0.87      0.88      0.87      6507

---------------------------------
floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5973
           1       0.91      0.42      0.58       534

    accuracy                           0.95      6507
   macro avg       0.93      0.71      0.78      6507
weighted avg       0.95      0.95      0.94      6507

---------------------------------
storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      5883
           1       0.76      0.47      0.58       624

    accuracy                           0.94      6507
   macro avg       0.86      0.73      0.77    

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
def build_model():
    pipeline = Pipeline(steps = [('vect', CountVectorizer(tokenizer=tokenize)),
                             ('tfidf', TfidfTransformer()),
                            ('classifier', MultiOutputClassifier( LogisticRegression()
                                                                        ))])
    parameters = {
                  'classifier__estimator__penalty' : ['l1','l2'],
                  'classifier__estimator__C':[0.2, 0.6, 1, 3, 5],
                  'classifier__estimator__solver' : ['liblinear', 'saga'],
                }
    
    rs = RandomizedSearchCV(pipeline, param_distributions = parameters,
                            n_iter = 10, scoring = "accuracy", random_state = 42,
                            n_jobs = -1,  return_train_score = True, verbose = 3)

    return rs

In [ ]:
model_lr = build_model()

In [ ]:
model_lr.get_params

In [ ]:
model_lr.fit(X_train, y_train)

In [ ]:
y_pred = model_lr.predict(X_test)

In [ ]:
model_lr.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
evaluate_model(model_lr, X_test, y_test, y_test.columns)

In [ ]:
evaluate_model(model, X_test, y_test, y_test.columns)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### List of additionally applied approaches

- Added StartingVerbExtractor as a new feature

1. GridSearchCv of StartingVerbExtractor+TFIDF and RandomForest
2. RandomizedSearchCv TFIDF and RandomForest
3. RandomizedSearchCv TFIDF and XgBoost



In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = word_tokenize(sentence)

            # index pos_tags to get the first word and part of speech tag
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)
    

In [ ]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, 
                      param_grid=parameters,
                      cv = 5,
                      verbose=3)

    return cv


In [ ]:
model_rf_2 = build_model()

In [ ]:
model_rf_2.fit(X_train, y_train)

In [ ]:
def build_model2():
    pipeline = Pipeline([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                            ])

    parameters = {
        #'cv':[4,5,6],
        'text_pipeline__vect__ngram_range':((1, 1), (1, 2), (2,2), (1,3)),
        'text_pipeline__vect__max_df':(0.5, 0.75, 1.0),
        'text_pipeline__vect__max_features': (None, 5000, 10000, 20000),
        'text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200, 500],
        'clf__estimator__min_samples_split': [2, 3, 4, 5, 6]
    }

    
    rs = RandomizedSearchCV(pipeline, param_distributions = parameters,
                            n_iter = 10, scoring = "accuracy", random_state = 42,
                            n_jobs = -1,  return_train_score = True, verbose = 3)

    return rs


In [ ]:
def build_model_xgb():
    pipeline = Pipeline([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ('clf', MultiOutputClassifier(XGBClassifier() ) )
                            ])

    parameters = {
        #'cv':[4,5,6],
        'text_pipeline__vect__ngram_range':((1, 1), (1, 2), (2,2), (1,3)),
        'text_pipeline__vect__max_df':(0.5, 0.75, 1.0),
        'text_pipeline__vect__max_features': (None, 5000, 10000, 20000),
        'text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200, 500],
        'clf__estimator__min_samples_split': [2, 3, 4, 5, 6]
    }

    
    rs = RandomizedSearchCV(pipeline, param_distributions = parameters,
                            n_iter = 10, scoring = "accuracy", random_state = 42,
                            n_jobs = -1,  return_train_score = True, verbose = 3)

    return rs


model_xgb = build_model_xgb()
model_xgb.fit(X_train, y_train)

In [29]:
def build_model():
    pipeline = Pipeline([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                            ])

    parameters = {
        #'cv':[4,5,6],
        #'text_pipeline__vect__ngram_range':((1, 1), (1, 2), (2,2), (1,3)),
        #'text_pipeline__vect__max_df':(0.5, 0.75, 1.0),
        #'text_pipeline__vect__max_features': (None, 5000, 10000, 20000),
        'text_pipeline__tfidf__use_idf': (True, False),
        #'clf__estimator__n_estimators': [50, 100, 200, 500],
#         'clf__estimator__min_samples_split': [5]
    }

    
    gs = GridSearchCV( pipeline, param_grid = parameters,
                         scoring = "accuracy", cv = 2,
                        n_jobs = -1,  return_train_score = True, verbose = 3)

    return gs

In [30]:
model_rf_gs = build_model()

In [31]:
model_rf_gs.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('text_pipeline',
                                        Pipeline(steps=[('vect',
                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fbd71c52c10>)),
                                                        ('tfidf',
                                                         TfidfTransformer())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'text_pipeline__tfidf__use_idf': (True, False)},
             return_train_score=True, scoring='accuracy', verbose=3)

In [35]:
evaluate_model(model_rf_gs, X_test, y_test,columns = Y.columns)


---------------------------------##-----------------
related
              precision    recall  f1-score   support

           0       0.73      0.40      0.51      1569
           1       0.83      0.95      0.89      4938

    accuracy                           0.82      6507
   macro avg       0.78      0.68      0.70      6507
weighted avg       0.81      0.82      0.80      6507

---------------------------------##-----------------
request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5403
           1       0.81      0.51      0.63      1104

    accuracy                           0.90      6507
   macro avg       0.86      0.74      0.78      6507
weighted avg       0.89      0.90      0.89      6507

---------------------------------##-----------------
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

 

/Users/azatgabdolla/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/azatgabdolla/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/azatgabdolla/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6226
           1       0.00      0.00      0.00       281

    accuracy                           0.96      6507
   macro avg       0.48      0.50      0.49      6507
weighted avg       0.92      0.96      0.94      6507

---------------------------------##-----------------
weather_related
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4693
           1       0.85      0.71      0.77      1814

    accuracy                           0.88      6507
   macro avg       0.87      0.83      0.85      6507
weighted avg       0.88      0.88      0.88      6507

---------------------------------##-----------------
floods
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5973
           1       0.90      0.49      0.63       534

    accuracy                           0.95      6507

In [76]:

def build_model():
    
    '''
    - Pipeline tfidf vectorizer + Multioutput XGB classifier
    
    - Introduction of parameters grid (many of them commented out to save running time)
    
    -  GridsearhCV Initialisation 
    '''
    pipeline = Pipeline([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('clf', MultiOutputClassifier(XGBClassifier() ) )
        
                            ])

    parameters = {
#         'text_pipeline__vect__ngram_range':((1, 1), (1, 2), (2,2), (1,3)),
#         'text_pipeline__vect__max_df':(0.5, 0.75, 1.0),
        'text_pipeline__vect__max_features': (None, 5000, 10000),
        'text_pipeline__tfidf__use_idf': (True, False)
#         'clf__estimator__n_estimators': [50, 100, 200, 500],
#         'clf__estimator__min_samples_split': [2, 3, 4, 5, 6]
    }


    gs = GridSearchCV( pipeline, param_grid = parameters,
                         scoring = "accuracy", cv = 5,
                        n_jobs = -1,  return_train_score = True, verbose = 3)

    return gs


In [77]:
model_xgb = build_model()

In [78]:
model_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: 

### Summary of applied methods

Quick summary of applied approached
- Added StartingVerbExtractor as a new feature

0. Logistic Regression - Baseline

0. Randomized Search for Logistic Regression, F1 score is lower than Baseline

1. GridSearchCv of StartingVerbExtractor+TFIDF and RandomForest
    - StartingVerbExtractor is out of Index for many cases
    - Takes too long time to train while the best parameters provide more volatile F1 scores than Logistic regression
2. RandomizedSearchCv TFIDF and RandomForest
    - Lower F1 score than F1 score of Baseline Logistic regression
3. RandomizedSearchCv TFIDF and XgBoost
    - Lower F1 score than F1 score of Baseline Logistic regression

Update: use of GridSearchCV is requirements, that is why randomized search and pipeline without parameters tuning are disregarded




### 9. Export your model as a pickle file

In [36]:
pickle.dump(model_rf_gs, open('model_final.pkl', 'wb'))



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.